In [0]:
from pyspark.sql.types import *

schema = StructType([
    StructField("date", StringType(), True),
    StructField("state", StringType(), True),
    StructField("district", StringType(), True),
    StructField("pincode", StringType(), True),
    StructField("age_0_5", IntegerType(), True),
    StructField("age_5_17", IntegerType(), True)
])

df1 = spark.read.csv(
    "/Volumes/workspace/aadhar/aadhar_bio/api_data_aadhar_biometric/api_data_aadhar_biometric_0_500000.csv",
    header=True,
    schema=schema,
)


In [0]:
schema = StructType([
    StructField("date", StringType(), True),
    StructField("state", StringType(), True),
    StructField("district", StringType(), True),
    StructField("pincode", StringType(), True),
    StructField("age_0_5", IntegerType(), True),
    StructField("age_5_17", IntegerType(), True)
])

df2 = spark.read.csv(
    "/Volumes/workspace/aadhar/aadhar_bio/api_data_aadhar_biometric/api_data_aadhar_biometric_500000_1000000.csv",
    header=True,
    schema=schema,
)

In [0]:
schema = StructType([
    StructField("date", StringType(), True),
    StructField("state", StringType(), True),
    StructField("district", StringType(), True),
    StructField("pincode", StringType(), True),
    StructField("age_0_5", IntegerType(), True),
    StructField("age_5_17", IntegerType(), True)
])

df3 = spark.read.csv(
    "/Volumes/workspace/aadhar/aadhar_bio/api_data_aadhar_biometric/api_data_aadhar_biometric_1000000_1500000.csv",
    header=True,
    schema=schema,
)

In [0]:
from pyspark.sql.types import *

schema = StructType([
    StructField("date", StringType(), True),
    StructField("state", StringType(), True),
    StructField("district", StringType(), True),
    StructField("pincode", StringType(), True),
    StructField("age_0_5", IntegerType(), True),
    StructField("age_5_17", IntegerType(), True)
])

df4 = spark.read.csv(
    "/Volumes/workspace/aadhar/aadhar_bio/api_data_aadhar_biometric/api_data_aadhar_biometric_1500000_1861108.csv",
    header=True,
    schema=schema,
)


In [0]:
df=df1.union(df2).union(df3).union(df4)
df.count()

1861108

In [0]:
from pyspark.sql.functions import month, to_date

df.select(month(to_date("date", "dd-MM-yyyy"))).distinct().display()

"month(to_date(date, dd-MM-yyyy))"
4
3
5
6
7
9
10
11
12


In [0]:
df.dropna("all")
df.count()

1861108

In [0]:

df.dropna("any")
df.count()

1861108

In [0]:
from pyspark.sql.functions import col

dup_df = (
    df.groupBy(
        "date",
        "state",
        "district",
        "pincode",
        "age_0_5",
        "age_5_17"
    )
    .count()
    .filter(col("count") > 1)
)

dup_df.count()


94896

In [0]:
df=df.dropDuplicates(["date","state","district","pincode","age_0_5","age_5_17"])
df.count()

1766212

In [0]:
df.select("state").distinct().display()

state
Andaman and Nicobar Islands
Kerala
Orissa
Andhra Pradesh
Sikkim
Daman & Diu
Goa
Dadra & Nagar Haveli
Dadra and Nagar Haveli
Pondicherry


In [0]:
from pyspark.sql.functions import col, lower, trim, regexp_replace

df_norm = (
    df.withColumn("state_clean",
        lower(trim(col("state")))
    )
    .withColumn(
        "state_clean",
        regexp_replace(col("state_clean"), "\\s+", " ")
    )
    .withColumn(
        "state_clean",
        regexp_replace(col("state_clean"), "&", "and")
    )
)

In [0]:
from pyspark.sql.functions import *

df_std = (
    df_norm
    .withColumn(
        "state_final",
        when(col("state_clean").isin(
            "west bengal", "westbengal", "west bangal"
        ), "West Bengal")
        .when(col("state_clean").isin(
            "odisha", "orissa"
        ), "Odisha")
        .when(col("state_clean").isin(
            "andhra pradesh"
        ), "Andhra Pradesh")
        .when(col("state_clean").isin(
            "tamil nadu", "tamilnadu"
        ), "Tamil Nadu")
        .when(col("state_clean").isin(
            "chhattisgarh", "chhatisgarh"
        ), "Chhattisgarh")
        .when(col("state_clean").isin(
            "uttaranchal"
        ), "Uttarakhand")
        .when(col("state_clean").isin(
            "pondicherry"
        ), "Puducherry")
        .  when(
            lower(trim(col("state"))).isin(
                "dadra and nagar haveli",
                "dadra & nagar haveli",
                "daman & diu",
                "daman and diu",
                "dadra and nagar haveli and daman and diu"
            ),
            "Dadra and Nagar Haveli and Daman and Diu"
        )
        .when(col("state_clean").isin(
            "jammu & kashmir",
            "jammu and kashmir"
        ), "Jammu and Kashmir")
        .otherwise(initcap(col("state_clean")))
    )
)

In [0]:
df_clean = df_std.drop("state", "state_clean").withColumnRenamed("state_final", "state")

In [0]:
df_clean.select("state").distinct().display()


state
Kerala
Andhra Pradesh
Sikkim
Goa
Himachal Pradesh
Haryana
Lakshadweep
Rajasthan
Chandigarh
Odisha


In [0]:
df_clean.count()

1766212

In [0]:

df_clean.orderBy("state","month","date").coalesce(1).write.mode("overwrite").csv("/Volumes/workspace/aadhar/aadhar_bio/clean_states_data_order")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5825031592920768>, line 1
----> 1 df_clean.orderBy("state","month","date").coalesce(1).write.mode("overwrite").csv("/Volumes/workspace/aadhar/aadhar_bio/clean_states_data_order")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:831, in DataFrameWriter.csv(self, path, mode, compression, sep, quote, escape, header, nullValue, escapeQuotes, quoteAll, dateFormat, timestampFormat, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, charToEscapeQuoteEscaping, encoding, emptyValue, lineSep)
    812 self.mode(mode)
    813 self._set_opts(
    814     compression=compression,
    815     sep=sep,
   (...)
    829     lineSep=lineSep,
    830 )
--> 831 self.format("csv").save(path)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:703, in DataFrameWriter.sav

In [0]:
df_clean.repartition("state") \
        .write \
        .mode("overwrite") \
        .csv("/Volumes/workspace/aadhar/aadhar_demo/clean_states_data")


In [0]:
df_clean.show()

In [0]:
from pyspark.sql.functions import col, month, to_date

df_clean.withColumn("month", month(to_date(col("date"), "yyyy-MM-dd"))).display()

In [0]:
df_clean.orderBy("state","month").display()


In [0]:
df_clean.show(150)

In [0]:
df_clean.orderBy("state", "month").coalesce(1).write.mode("overwrite").option("header", "true").csv("/Volumes/workspace/aadhar/aadhar_bio/sorted_clean_states_data_bio")
df_clean.count()

In [0]:
df_clean.write.mode("overwrite").partitionBy("state").option("header", "true").csv("/Volumes/workspace/aadhar/aadhar_bio/clean_states_data")


In [0]:
df_all_state = (
    df_clean
    .groupBy("state", "month")
    .agg(
        sum("age_0_5").alias("sum_age_0_5"),
        sum("age_5_17").alias("sum_age_5_17")
    )
    .orderBy("state", "month")
)

df_all_state.display()

In [0]:
df_all_state.write.mode("overwrite").csv("/Volumes/workspace/aadhar/aadhar_bio/all_states_data_bio")
df_all_state.count()

In [0]:
df_all_state.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("/Volumes/workspace/aadhar/aadhar_bio/all_states_data")

In [0]:
df_all_state.display()

#Cleaning State Distict

In [0]:
df=spark.read.csv("/Volumes/workspace/aadhar/aadhar_bio/sorted_clean_states_data_bio/", header=True)
df.display()
df.count()


date,district,pincode,age_0_5,age_5_17,state,month
01-03-2025,Andamans,744101,16,193,Andaman And Nicobar Islands,3
01-03-2025,Nicobar,744303,46,27,Andaman And Nicobar Islands,3
01-03-2025,South Andaman,744112,11,10,Andaman And Nicobar Islands,3
01-03-2025,Nicobar,744304,16,14,Andaman And Nicobar Islands,3
01-03-2025,North And Middle Andaman,744205,122,69,Andaman And Nicobar Islands,3
01-03-2025,South Andaman,744106,49,30,Andaman And Nicobar Islands,3
01-03-2025,South Andaman,744107,42,18,Andaman And Nicobar Islands,3
01-03-2025,South Andaman,744104,17,12,Andaman And Nicobar Islands,3
01-03-2025,South Andaman,744206,55,26,Andaman And Nicobar Islands,3
01-03-2025,North And Middle Andaman,744204,103,76,Andaman And Nicobar Islands,3


1766212

In [0]:
df.select("district").distinct().display()

district
Kamrup
chittoor
Siwan
Nellore
Nalbari
Dhemaji
Marigaon
Anantapur
Jorhat
Golaghat


In [0]:
from pyspark.sql.functions import col, trim, lower, regexp_replace, initcap
df_clean = df.withColumn(
    "district",
    initcap(
        regexp_replace(
            trim(lower(col("district"))),
            "\\s+",
            " "
        )
    )
)


In [0]:
df_clean.select("district").distinct().count()


950

In [0]:
df_state_district_count = (
    df_clean
    .groupBy("state")
    .agg(countDistinct("district").alias("district_count"))
    .orderBy("state")
)

In [0]:
df_state_district_count.display()

state,district_count
Andaman And Nicobar Islands,5
Andhra Pradesh,45
Arunachal Pradesh,25
Assam,37
Bihar,46
Chandigarh,3
Chhattisgarh,40
Dadra and Nagar Haveli and Daman and Diu,4
Delhi,13
Goa,4


In [0]:
from pyspark.sql import Row

official_data = [
    ("Andhra Pradesh", 26),
    ("Arunachal Pradesh", 26),
    ("Assam", 35),
    ("Bihar", 38),
    ("Chhattisgarh", 33),
    ("Goa", 2),
    ("Gujarat", 33),
    ("Haryana", 22),
    ("Himachal Pradesh", 12),
    ("Jharkhand", 24),
    ("Karnataka", 31),
    ("Kerala", 14),
    ("Madhya Pradesh", 55),
    ("Maharashtra", 36),
    ("Manipur", 16),
    ("Meghalaya", 12),
    ("Mizoram", 11),
    ("Nagaland", 16),
    ("Odisha", 30),
    ("Punjab", 23),
    ("Rajasthan", 50),
    ("Sikkim", 6),
    ("Tamil Nadu", 38),
    ("Telangana", 33),
    ("Tripura", 8),
    ("Uttar Pradesh", 75),
    ("Uttarakhand", 13),
    ("West Bengal", 23),
    ("Andaman And Nicobar Islands", 3),
    ("Chandigarh", 1),
    ("Dadra and Nagar Haveli and Daman and Diu", 3),
    ("Delhi", 11),
    ("Jammu and Kashmir", 20),
    ("Ladakh", 2),
    ("Lakshadweep", 1),
    ("Puducherry", 4)
]

official_df = spark.createDataFrame(
    official_data,
    ["state", "official_district_count"]
)


In [0]:
from pyspark.sql.functions import countDistinct

your_counts = (
    df_clean
    .groupBy("state")
    .agg(countDistinct("district").alias("your_district_count"))
)


In [0]:
from pyspark.sql.functions import abs, when, col
comparison = (
    official_df
    .join(your_counts, on="state", how="left")
    .withColumn(
        "difference",
        abs(col("your_district_count") - col("official_district_count"))
    )
    .withColumn(
        "status",
        when(col("difference") == 0, "MATCH")
        .when(col("difference") <= 2, "CLOSE")
        .otherwise("MISMATCH")
    )
    .orderBy("difference", ascending=False)
)


In [0]:
# View data
comparison.display()

# Write data
(
    comparison
    .write
    .mode("overwrite")
    .option("header", "true")
    .csv("/Volumes/workspace/aadhar/aadhar_bio/aadhar_district_comparison")
)


state,official_district_count,your_district_count,difference,status
West Bengal,23,47,24,MISMATCH
Karnataka,31,53,22,MISMATCH
Andhra Pradesh,26,45,19,MISMATCH
Maharashtra,36,52,16,MISMATCH
Uttar Pradesh,75,90,15,MISMATCH
Odisha,30,41,11,MISMATCH
Jharkhand,24,33,9,MISMATCH
Bihar,38,46,8,MISMATCH
Jammu and Kashmir,20,28,8,MISMATCH
Chhattisgarh,33,40,7,MISMATCH
